# Grover's Algorithm in a n$^2$ x n$^2$ search space

   Consider a 4x4 Sudoku with 16 cells and 2 qubits per cell

Use Grover's algorithm to solve a 4x4 Sudoku puzzle.
Grover algorithm consist of four main steps:
   1. The initial state preparation. Identify the number of qubits. Apply a Hadamard transformation (|$\psi$$\rangle$: H($|0 \rangle$$^\otimes$$^n$|1$\rangle$) to each qubit, putting the qubits in a state of equal superposition.
   2. Implementation of an oracle ('black box') and a Grover operator. The oracle recognizes the solution. This allows us to mark the target states in N possible states in the dataset.
   3. Application of an amplitude amplification operation, or a diffuser. The diffuser takes the state marked by the oracle and increases its probability of measuring the correct solution. The marked state will be described by a 'negative' value. The diffuser will detect the state with the phase difference.
4. Measure and repeat steps 2 and 3 (($\pi$/4)$\sqrt{N}$) and observe the solution emerge with high probability

In [ ]:
#Brief Summary Below:

In [ ]:
#Import necessary packages
#Consider a 4x4 Sudoku puzzle shape using '*' for empty cells
#Define a list of constraints and parameters based on the rules of Sudoku
#Build an oracle (without the use of controlled Z-gates)
#Build a diffuser
#Show Grover's circuit (oracle and diffuser)

In [20]:
from qiskit.circuit import QuantumCircuit, QuantumRegister, AncillaRegister
from qiskit.quantum_info import Statevector, Operator

import matplotlib.pyplot as plt
from itertools import combinations
from math import pi, ceil, log2, floor, sqrt
import numpy as npv
from qiskit.quantum_info import Operator
from qiskit import QuantumCircuit
import numpy as np

In [21]:
#Consider a 4x4 Sudoku puzzle shape using * for empty cells

n = 2
ExampleA = np.array([2, 0, 3, 1, 1, 3, 2, '*', 0, 2, 1, 3, '*', 1, 0, 2])
ExampleA = ExampleA.reshape(4, 4)

ExampleA

array([['2', '0', '3', '1'],
       ['1', '3', '2', '*'],
       ['0', '2', '1', '3'],
       ['*', '1', '0', '2']], dtype='<U21')

In [22]:
#Identify empty (*) cells 

def empty(ExampleA):
    missing = []
    for i in range(len(ExampleA)):
        for j in range(len(ExampleA)):
            if ExampleA[i][j] == '*':
                missing.append([i, j])
    return missing
    
#Count the number of * in this case

missing_positions = empty(ExampleA)
num_missing = len(missing_positions)
print(f"Number of unknowns {num_missing}")

Number of unknowns 2


In [23]:
missing_positions

[[1, 3], [3, 0]]

In [24]:
#Define a valid Sudoku puzzle (the search space)

def possible_candidate(ExampleA, empty, candidate):
    new_puzzle = [row.copy() for row in puzzle]
    for (i, j), val in zip(empty, candidate):
        new_puzzle[i][j] = val
    return new_puzzle

 
#4x4 matrix containing empty (*), or missing values, and numbers 0, 1, 2 and 3. Each matrix element is represented by two bits

#n^2= 4; block_size = 2

#Constraints on rows, columns and boxes with 3(n^2 -1) constraints per blank

n=2
def confirm_sudoku_constraints(ExampleA):

    constraints = []

    for i in range(len(missing_positions)):
        row = missing_positions[i][0]
        column = missing_positions[i][1]
      
    #Row constraints
        for j in range(n**2):
          if j != column:
            if ExampleA[row][j] == '*':
              constraints.append([2*i, 2*missing_positions.index([row, j]) ])
            else:
              constraints.append([2*i, ExampleA[row][j]])
            
    #Column constraints
        for j in range(n**2):
          if j != row:
            if ExampleA[j][column] == '*':
              constraints.append([2*i, 2*missing_positions.index([j, column]) ])
            else:
              constraints.append([2*i, ExampleA[j][column]])
            
    #Box (2x2 sub-grid) constraints
        if row < n:
          if column < n:
            for j in range(n):
              for k in range(n):
                if ExampleA[j][k] == '*':
                  constraints.append([2*i, 2*missing_positions.index([j, k]) ])
                else:
                  constraints.append([2*i, ExampleA[j][k]])
          elif column >= n:
            for j in range(n):
              for k in range(n):
                if ExampleA[j][k + n] == '*':
                  constraints.append([2*i, 2*missing_positions.index([j, k+n]) ])
                else:
                  constraints.append([2*i, ExampleA[j][k+n]])
        elif row >= n:
          if column < n:
            for j in range(n):
              for k in range(n):
                if ExampleA[j + n][k] == '*':
                  constraints.append([2*i, 2*missing_positions.index([j + n, k]) ])
                else:
                  constraints.append([2*i, ExampleA[j + n][k]])
          elif column >= n:
            for j in range(n):
              for k in range(n):
                if ExampleA[j + n][k + n] == '*':
                  constraints.append([2*i, 2*missing_positions.index([j + n, k+n]) ])
                else:
                  constraints.append([2*i, ExampleA[j + n][k+n]])

#Remove duplicate entries
    temp = []
    for i in range(len(constraints)):
        if constraints[i] not in temp and constraints[i][0] != constraints[i][1]:
            temp.append(constraints[i])

    constraints = temp 
 

    return constraints

#work in progress

In [25]:
confirm_sudoku_constraints(ExampleA)

[[0, '1'], [0, '3'], [0, '2'], [2, '1'], [2, '0'], [2, '2']]

In [26]:
#Map candidate list to a quantum register.

#State preparation (where q represents the puzzle bits)
QuReg = QuantumRegister(size=4, name = "q");
q_qc = QuantumCircuit(QuReg, name = "State Prep");
q_qc.h(QuReg);
Initial_Circ = q_qc.to_gate();
q_qc.draw()

┌───┐
q_0: ┤ H ├
     ├───┤
q_1: ┤ H ├
     ├───┤
q_2: ┤ H ├
     ├───┤
q_3: ┤ H ├
     └───┘

In [27]:
#Build an oracle 

#Use Sudoku rules as described above and implement a 'negative' phase marker (i.e. flip the phase of a valid candidate)

#Assume that the data is labeled as n-bit boolean strings in {0,1}$^n$ rather than being indexed from 1 to N.

#Re-imagine constraints for nqubits


In [ ]:
def construct_oracle(nqubits, valid_candidate):
    

In [28]:
#Build a diffuser for n number of qubits ("general diffuser")

def diffuser(nqubits):
    qc = QuantumCircuit(nqubits)
    
    # Apply transformation (H-gates)
    for qubit in range(nqubits):
        qc.h(qubit)
        
    # Apply X-gates
    for qubit in range(nqubits):
        qc.x(qubit)
        
    # Multi-controlled-Z gate
    qc.h(nqubits-1)
    qc.mcx(list(range(nqubits-1)), nqubits-1)
    qc.h(nqubits-1)
    
    # Apply X-gate
    for qubit in range(nqubits):
        qc.x(qubit)
        
    # Apply H-gates
    for qubit in range(nqubits):
        qc.h(qubit)
        
    # Return diffuser as a gate
    D = qc.to_gate()
    D.name = "Diffuser"
    return Diffuser

In [29]:
QuReg = QuantumRegister(size=4, name = "q");
Diff_qc = QuantumCircuit(QuReg, name = "Diffuser");
Diff_qc.h(QuReg);
Diff_qc.x(QuReg);
Diff_qc.mcx(QuReg[0:-1], QuReg[-1])
Diff_qc.x(QuReg);
Diff_qc.h(QuReg);

Diff_Circ = Diff_qc.to_gate();
Diff_qc.draw()

┌───┐┌───┐     ┌───┐┌───┐
q_0: ┤ H ├┤ X ├──■──┤ X ├┤ H ├
     ├───┤├───┤  │  ├───┤├───┤
q_1: ┤ H ├┤ X ├──■──┤ X ├┤ H ├
     ├───┤├───┤  │  ├───┤├───┤
q_2: ┤ H ├┤ X ├──■──┤ X ├┤ H ├
     ├───┤├───┤┌─┴─┐├───┤├───┤
q_3: ┤ H ├┤ X ├┤ X ├┤ X ├┤ H ├
     └───┘└───┘└───┘└───┘└───┘

In [31]:
#Determine the number of Grover iterations
iterations = int(np.floor(pi / 4 * np.sqrt(possible_candidate)))

TypeError: loop of ufunc does not support argument 0 of type function which has no callable sqrt method

In [32]:
#Ouput a final solution
solution_puzzle = fill_candidate(PuzzleA, missing_positions, candidate_list[found_index])

# Output only the valid solution
print("Valid 4x4 Sudoku solution:")
for row in solution_puzzle:
    print(row)

NameError: name 'fill_candidate' is not defined